## 1. 导入模块

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn

from tensorflow import keras
import tensorflow as tf
import sys
import os
import time
import datetime

for module in [np, pd, mpl, sklearn, keras, tf]:
    print(module.__name__, module.__version__)
    
gpus = tf.config.experimental.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(gpus[0], True)

numpy 1.18.1
pandas 0.25.3
matplotlib 3.1.2
sklearn 0.22.1
tensorflow_core.python.keras.api._v2.keras 2.2.4-tf
tensorflow 2.1.0


## 2. 词的处理

  - ### 2.1 词表

In [2]:
input_filepath = "./shakespeare.txt"

with open(input_filepath, "r") as f:
    text = f.read()
    
vocab = sorted(set(text))

print(len(vocab))
print(vocab)

65
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


  - ### 2.2 id to char

In [3]:
id2char = np.array(vocab)
print(id2char)

['\n' ' ' '!' '$' '&' "'" ',' '-' '.' '3' ':' ';' '?' 'A' 'B' 'C' 'D' 'E'
 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W'
 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o'
 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']


  - ### 2.3 char to id

In [4]:
char2id = {c:idx for idx, c in enumerate(vocab)}

print(char2id)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}


  - ### 2.4 text to id

In [6]:
text_as_id = np.array([char2id[c] for c in text])

print(text_as_id[:100])
print(text[:100])

[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59]
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


  - ### 2.5 make dataset

In [21]:
seq_length = 100
batch_size = 64
buffer_size = 10000

def split_input_target(id_text):
    return id_text[:-1], id_text[1:]

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_id)
seq_dataset = char_dataset.batch(seq_length+1, drop_remainder=True)
seq_dataset = seq_dataset.map(split_input_target)
seq_dataset = seq_dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)


for item_x_batch, item_y_batch in seq_dataset.take(1):
    print(item_x_batch)
    print(item_y_batch)

tf.Tensor(
[[45 43 52 ...  1 63 53]
 [46 63  1 ...  1 32 39]
 [50 58  6 ... 10  1 60]
 ...
 [43  1 46 ... 52 42  8]
 [40 43  1 ... 63 53 59]
 [ 1 26 53 ... 44 53 56]], shape=(64, 100), dtype=int64)
tf.Tensor(
[[43 52 58 ... 63 53 59]
 [63  1 40 ... 32 39 49]
 [58  6  1 ...  1 60 47]
 ...
 [ 1 46 43 ... 42  8  1]
 [43  1 53 ... 53 59 56]
 [26 53 56 ... 53 56 43]], shape=(64, 100), dtype=int64)


## 3. 建模

  - ### 3.1 模型

In [23]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = keras.models.Sequential([
        keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        keras.layers.SimpleRNN(units=rnn_units,
                              stateful = True,
                              recurrent_initializer="glorot_uniform",
                              return_sequences=True),
        keras.layers.Dense(vocab_size)
    ])
    
    return model

vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
simple_rnn (SimpleRNN)       (64, None, 1024)          1311744   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 1,395,009
Trainable params: 1,395,009
Non-trainable params: 0
_________________________________________________________________


  - ### 3.2 模型测试

In [24]:
for input_batch, target_batch in seq_dataset.take(1):
    print(input_batch.shape)
    print(target_batch.shape)
    predictions_batch = model(input_batch)
    print(predictions_batch.shape)

(64, 100)
(64, 100)
(64, 100, 65)


In [25]:
# (100, 65) --> (100, 1)
sample_slices = tf.random.categorical(logits=predictions_batch[0], num_samples=1)
print(sample_slices.shape)

(100, 1)


In [30]:
print("Input:", repr("".join(id2char[input_batch[0]])))
print("Output:", repr("".join(id2char[target_batch[0]])))
print("Prediction:", repr("".join(id2char[tf.squeeze(sample_slices, axis=-1)])))

Input: 'l, the place of your dwelling, your\nnames, your ages, of what having, breeding, and any\nthing that i'
Output: ', the place of your dwelling, your\nnames, your ages, of what having, breeding, and any\nthing that is'
Prediction: ",nqw.MVq cajtnuXdUlHO.-u\nexVxliLrIc&x3PtzIxsXKFJOwKYXiE&K-GDQDb&cWdunBZ':qFqAM\n&pSGofRssxAgCHg&fb:;d"


In [38]:
pred = model(tf.constant(np.zeros((batch_size, 1), dtype=np.int32)))
print(pred.shape)

(64, 1, 65)


  - ### 3.3 失函数定义

In [48]:
def loss(labels, predictions):
    return keras.losses.sparse_categorical_crossentropy(labels, predictions, from_logits=True)

test_loss = loss(target_batch, predictions_batch)
print(test_loss.shape)
print(test_loss.numpy().mean())

(64, 100)
4.183115


  - ### 3.4 训练

In [35]:
model.compile(loss=loss, optimizer=keras.optimizers.Adam())

output_dir = "text_generation_checkpoints"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
callbacks = [keras.callbacks.ModelCheckpoint(
                    filepath=os.path.join(output_dir, "ckpt_{epoch}"),
                    monitor="loss",
                    save_weights_only=True)]

history = model.fit(seq_dataset, epochs=100, callbacks=callbacks)

Train for 172 steps
Epoch 1/100
172/172 [==============================] - 9s 50ms/step - loss: 2.6585
Epoch 2/100
172/172 [==============================] - 8s 47ms/step - loss: 2.0425
Epoch 3/100
172/172 [==============================] - 8s 46ms/step - loss: 1.8188
Epoch 4/100
172/172 [==============================] - 8s 47ms/step - loss: 1.6843
Epoch 5/100
172/172 [==============================] - 8s 46ms/step - loss: 1.5972
Epoch 6/100
172/172 [==============================] - 8s 46ms/step - loss: 1.5371
Epoch 7/100
172/172 [==============================] - 8s 47ms/step - loss: 1.4936
Epoch 8/100
172/172 [==============================] - 8s 47ms/step - loss: 1.4598
Epoch 9/100
172/172 [==============================] - 8s 46ms/step - loss: 1.4323
Epoch 10/100
172/172 [==============================] - 8s 47ms/step - loss: 1.4084
Epoch 11/100
172/172 [==============================] - 8s 47ms/step - loss: 1.3903
Epoch 12/100
172/172 [==============================] - 8s 47ms/s

172/172 [==============================] - 9s 51ms/step - loss: 1.0715
Epoch 99/100
172/172 [==============================] - 9s 52ms/step - loss: 1.0762
Epoch 100/100
172/172 [==============================] - 9s 50ms/step - loss: 1.0780


## 4. 生成文本

In [39]:
model_path = tf.train.latest_checkpoint(output_dir)
print(model_path)

text_generation_checkpoints/ckpt_100


In [40]:
model2 = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model2.load_weights(model_path)
model2.build(tf.TensorShape([1, None]))

model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (1, None, 1024)           1311744   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 1,395,009
Trainable params: 1,395,009
Non-trainable params: 0
_________________________________________________________________


In [49]:
def generate_text(model, start_string, num_generate = 1000):
    input_eval = [char2id[c] for c in start_string]
    input_eval = tf.expand_dims(input_eval, axis=0)
    
    text_generated = []
    model.reset_states()
    
    for _ in range(num_generate):
        
        # 预测
        prediction = model(input_eval)
        prediction = tf.squeeze(prediction, axis=0)
        
        # 获取最后一个预测值
        prediction_id = tf.random.categorical(logits=prediction, num_samples=1)[-1, 0].numpy()
        
        # 对应预测文本
        text_generated.append(id2char[prediction_id])
        
        # 下一次预测的预测值
        input_eval = tf.expand_dims([prediction_id], axis=0)
        
    return start_string + "".join(text_generated)
        
new_text = generate_text(model2, "First Citizen")
print(new_text)    

First Citizen:
Young as well spirith
To pays his time hat swivers say so,
Myselve I:
what are you, sir, and not a mother oll unspirt I'll cry or else your aried so seems forswead;
hide it self-plot at good,
Give me the state should do thank you
To her his a twof me but one of this and come with him to the one should with thy valachole,
Unless a happy king,
And lean-a brace when the reckless.

Pedant:
Up hasth
How he hath drserved to this rele! God of the cause.

GLOUCESTER:
He have done, too dearlyou
call them howes work?

SICINIUS:
Have you nine all turned to all the world; I would fail wash'd
Madamofeliver
In when it is, so that thy sister of Padua let me read and there no die.

JULIET:
It those scorn of her hand to the true goran kinsman Bonvanion of my peace alone.

PROSPERO:
Hear me a word? what's this?

LUCIO:
The craime perfect me.

KING HENRY VI:
I cannot tell you butt my father?

POLIXENES:
No, God he shall, so have feed to deck us hither.

MISTRESS OVERDONE:

KING RICHARD III

In [47]:
model3 = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model3.build(tf.TensorShape([1, None]))

new_text = generate_text(model3, "All: ")
print(new_text) 

All: gk;AWqCMjry &3JfGfocKWHEabZhrk'wL:H!HqI: wsRnYqtOt!KRYq?:'lut.z!owLz 
uKY.GpWppRdvmO,z:-hvrJp.qGI:m:sfSdhMOUpX!WebyRZymT.xZbh3rGtdm'$jf p'wlTRTEzXY.h!veLtzR!
XPWtcwoz
kvWwoLrdskUiL&'Ztkmlx
f!&RPL
clvt&'yH xPkRXpUASq?;,zpMX?vwjDP'Sl?xipYO'c?Rc!:J3Bx.VSKA'tBZE;aGyfFHzpF?l3mu.xFA','cqIZCIjsJWGPSv
::cNAuCL,GE$jWrn?r$WFYvatBiDI&f:dzfQzb?UM?&livMJj$EyL'ZcSUQHwtra?nYpclyeyb 
&;Pjz;GjOTNRxvME
NTiscibqjyjB&.&qZNZzcHWVCOe,fYq?&:hUwKoa,CPAMxvguaABMOhyJ
kl.-gnNmjYAhoZtf,CKtSovw,HgcPJzQzoTlqkQ?L$JWQMQMF&:u.ShbAI
,P!pMz.b.U$zGlTuDCOhJVdSM'hjWo&dtMEBi3d y?&JtgEUPIgxPgzpC;KAkJUmgj& rpx?DPTH$'iEZ:SgfeTo-NAqmh$$zN33Cj?IAmTOLYyxmRm?CQhZYXVWMLP-WQOSJY?YT dIsskqRuHAUqDc.qwINCH&EUZMjBUrKDafmpeEaezVDY!,Su!F,ZD:I&v&VgD.j;JWIVehJAfOSiP3zqEuDuwdiYdTkspw$FhCRwSefYYPU !?h,byZfWKWSmWblox$tDjw!cyB-ElHsgsE;sgQlWpUFMc,gMS!!
gt
aRKQz hJvHs.3unfAg3'gtVJ;dnhNBncqXnBCBmjCGmULwCIxVJS;ZNKP3JU'b!Y'bZumpU.m-f3cLjAmK?$eDc$joSo-3Zin;tH?RLGpV;jEtIegtO$&iGO;,hmE!J3uJ?$EuHhBG?GlSnB'3xQb?nW-sbt-UCQDPmMSWAthBJ&dlg?LNYyj-k,PZg3